<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=357f34abbeee29a9e868d7919deccff1ba4cc3e83d5525060380b98f80522da7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-12 08:51:12
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.41 C
-------------------
Today PnL: 20.22 K (0.14%)
Current PnL: -17.72 L (-11.92%)
CY Booked + Current PnL: -6.69 L (-4.5%)
-------------------
Total profit:  2.03 L
Total loss:  -19.75 L
-------------------
Total Booked + Current PnL: 20.29 L (16.67%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.8%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.82 L (59.25%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.20,60.0,H-LC,11.20,173349.0,17172.0,8927.0,-0.62,11.00,5.15,16.71,22.0,1.92,1.26,30.35,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,61.0,M-LC,4.29,218267.0,16788.0,10433.0,0.67,8.33,4.78,13.51,60.0,1.61,1.59,21.98,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,104.70,60.0,M-SC,5.28,88183.0,-12594.0,12690.0,-0.51,-12.50,14.39,0.09,245.0,-0.99,0.64,15.85,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
41,ITC,452.00,-37.08,56.0,H-LC,3.01,202959.0,2821.0,18063.0,0.48,1.41,8.90,10.44,4.0,0.16,1.48,7.05,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,7.00,226994.0,2032.0,165615.0,1.62,0.90,72.96,74.53,192.0,0.01,1.65,33.45,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.34,40.0,H-MC,1.99,135250.0,-538.0,43294.0,-0.63,-0.40,32.01,31.48,107.0,-0.01,0.99,11.67,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.67,245606.0,-2125.0,77194.0,-0.06,-0.86,31.43,30.30,92.0,-0.03,1.79,12.32,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-31.08,50.0,H-MC,4.62,199575.0,-1245.0,70031.0,0.38,-0.62,35.09,34.25,80.0,-0.02,1.45,4.54,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302827.0,-13015.0,125673.0,-0.24,-4.12,41.50,35.67,5.0,-0.10,2.21,9.16,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.64,43.0,H-LC,13.62,267202.0,-163.0,65625.0,0.21,-0.06,24.56,24.48,48.0,-0.00,1.95,54.88,XY24,BTT,FINANCE
56,RELIANCE,1533.00,-13.66,48.0,H-LC,6.30,215795.0,5429.0,23349.0,0.46,2.58,10.82,13.68,37.0,0.23,1.57,19.53,XY25,NTT,REFINERIES
31,HCLTECH,1943.91,1.08,48.0,H-LC,10.16,230429.0,-11495.0,74774.0,0.26,-4.75,32.45,26.15,8.0,-0.15,1.68,8.89,X40,ATH,IT
80,UNITDSPR,1644.00,-13.52,49.0,H-LC,8.13,226868.0,-7276.0,55900.0,1.19,-3.11,24.64,20.77,86.0,-0.13,1.65,3.60,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.05,58.0,H-LC,1.26,238118.0,-8033.0,87246.0,-0.29,-3.26,36.64,32.18,16.0,-0.09,1.73,31.60,X200,ATH,IT
41,ITC,452.00,-37.08,56.0,H-LC,3.01,202959.0,2821.0,18063.0,0.48,1.41,8.90,10.44,4.0,0.16,1.48,7.05,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.20,63.0,H-LC,3.35,285667.0,20241.0,36537.0,0.27,7.63,12.79,21.40,11.0,0.55,2.08,16.00,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.78,52.0,H-LC,5.95,213830.0,9330.0,32096.0,-0.19,4.56,15.01,20.25,10.0,0.29,1.56,13.69,X40N,BTT,PHARMA
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302827.0,-13015.0,125673.0,-0.24,-4.12,41.50,35.67,5.0,-0.10,2.21,9.16,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.00,252.07,78.0,M-SC,6.12,157209.0,-25869.0,104843.0,18.09,-14.13,66.69,43.14,230.0,-0.25,1.15,47.88,XY24,NTT,MISC
57,REPCOHOME,880.00,-59.76,55.0,H-SC,7.90,181705.0,-54038.0,243339.0,4.14,-22.92,133.92,80.30,134.0,-0.22,1.32,23.01,XY24,NTT,FINANCE
32,HINDZINC,730.22,18.93,59.0,M-LC,1.77,194119.0,-10957.0,123518.0,3.10,-5.34,63.63,54.89,52.0,-0.09,1.41,17.47,X5K,ATH,METALS
14,BERGEPAINT,680.00,-16.95,55.0,H-MC,6.79,223297.0,-4042.0,52095.0,2.93,-1.78,23.33,21.14,106.0,-0.08,1.63,25.86,XY24,NTT,PAINTS
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.0,2.55,32.0,H-SC,7.10,181712.0,-9294.0,66997.0,-5.16,-4.87,36.87,30.21,157.0,-0.14,1.32,15.00,X40N,NTT,FINANCE
82,VALIANTORG,838.0,-323.33,53.0,H-SC,8.39,124100.0,-39505.0,160821.0,-3.27,-24.15,129.59,74.15,139.0,-0.25,0.90,59.08,XR,NTT,CHEMICALS
85,WHIRLPOOL,2270.0,-41.04,51.0,M-SC,1.98,99750.0,8252.0,70503.0,-2.91,9.02,70.68,86.07,223.0,0.12,0.73,44.85,XR,NTT,DURABLES
8,ATULAUTO,844.0,3570.27,56.0,M-SC,7.49,114366.0,-33002.0,84814.0,-2.39,-22.39,74.16,35.16,236.0,-0.39,0.83,16.81,XY24,NTT,AUTO
81,VAIBHAVGBL,521.0,51.52,59.0,H-SC,10.24,131480.0,-51295.0,162365.0,-2.30,-28.06,123.49,60.77,125.0,-0.32,0.96,17.76,XR,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,101.87,64.0,H-SC,12.69,127548.0,-14931.0,34451.0,-2.30,-10.48,27.01,13.70,163.0,-0.43,0.93,50.54,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.70,60.0,M-SC,5.28,88183.0,-12594.0,12690.0,-0.51,-12.50,14.39,0.09,245.0,-0.99,0.64,15.85,OX40N,NTT,DURABLES
19,CERA,9475.0,-20.05,46.0,H-SC,2.23,145854.0,-30049.0,72066.0,-0.51,-17.08,49.41,23.89,149.0,-0.42,1.06,25.96,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-19.17,56.0,H-MC,8.05,109298.0,-22537.0,64759.0,-1.56,-17.09,59.25,32.03,98.0,-0.35,0.80,22.42,OX40N,NTT,IT
44,KANSAINER,340.0,-67.79,57.0,H-SC,2.28,221949.0,-47718.0,84052.0,0.95,-17.70,37.87,13.47,138.0,-0.57,1.62,12.26,XY24,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,54.0,H-LC,13.70,293675.0,-52281.0,176146.0,0.46,-15.11,59.98,35.80,1.0,-0.30,2.14,4.04,X40,BTT,IT
39,INFY,2275.00,-17.44,53.0,H-LC,9.75,321566.0,8678.0,163002.0,-1.49,2.77,50.69,54.87,3.0,0.05,2.34,9.61,X40,BTT,IT
41,ITC,452.00,-37.08,56.0,H-LC,3.01,202959.0,2821.0,18063.0,0.48,1.41,8.90,10.44,4.0,0.16,1.48,7.05,X40,NTT,FMCG
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302827.0,-13015.0,125673.0,-0.24,-4.12,41.50,35.67,5.0,-0.10,2.21,9.16,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,53.0,H-LC,12.28,250469.0,-11150.0,130369.0,1.09,-4.26,52.05,45.57,7.0,-0.09,1.82,6.58,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,7.00,226994.0,2032.0,165615.0,1.62,0.90,72.96,74.53,192.0,0.01,1.65,33.45,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.74,188680.0,24010.0,43962.0,-0.15,14.58,23.30,41.28,89.0,0.55,1.37,23.73,X40N,ATH,AC
84,VOLTAS,1918.49,-0.24,55.0,H-MC,3.16,209295.0,17553.0,78486.0,0.03,9.15,37.50,50.08,99.0,0.22,1.52,16.57,XY25,ATH,AC
24,DABUR,735.00,-2.58,60.0,H-MC,6.00,212199.0,15171.0,74461.0,-0.62,7.70,35.09,45.49,102.0,0.20,1.55,21.74,XY24,BTT,FMCG
18,CAMS,5211.76,-5.16,51.0,H-SC,4.81,107814.0,5810.0,38112.0,-1.04,5.70,35.35,43.06,122.0,0.15,0.79,23.79,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
36,INDIAMART,4850.92,-50.35,55.0,H-SC,12.17,138457.0,15121.0,118644.0,1.40,12.26,85.69,108.45,119.0,0.13,1.01,37.35,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-41.04,51.0,M-SC,1.98,99750.0,8252.0,70503.0,-2.91,9.02,70.68,86.07,223.0,0.12,0.73,44.85,XR,NTT,DURABLES
86,WIPRO,420.00,-10.62,57.0,M-LC,6.26,157424.0,6479.0,102971.0,-0.72,4.29,65.41,72.51,53.0,0.06,1.15,10.57,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,64.0,H-MC,5.50,181638.0,1830.0,110581.0,0.52,1.02,60.88,62.52,88.0,0.02,1.32,35.45,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-5.35,44.0,H-SC,14.31,88150.0,-12813.0,107340.0,-0.91,-12.69,121.77,93.63,148.0,-0.12,0.64,31.30,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-41.65,56.0,L-SC,43.34,87686.0,-25863.0,65914.0,-2.10,-22.78,75.17,35.27,268.0,-0.39,0.64,114.61,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,56.0,H-SC,13.06,94770.0,-3210.0,24811.0,0.48,-3.28,26.18,22.05,152.0,-0.13,0.69,37.76,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
82,VALIANTORG,838.00,-323.33,53.0,H-SC,8.39,124100.0,-39505.0,160821.0,-3.27,-24.15,129.59,74.15,139.0,-0.25,0.90,59.08,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-41.65,56.0,L-SC,43.34,87686.0,-25863.0,65914.0,-2.10,-22.78,75.17,35.27,268.0,-0.39,0.64,114.61,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,3.01,82158.0,-20052.0,113444.0,0.14,-19.62,138.08,91.37,208.0,-0.18,0.60,33.37,XY25,NTT,FINANCE
27,GREENPANEL,537.0,252.07,78.0,M-SC,6.12,157209.0,-25869.0,104843.0,18.09,-14.13,66.69,43.14,230.0,-0.25,1.15,47.88,XY24,NTT,MISC
75,TITAGARH,1548.0,-3.85,62.0,H-SC,6.51,157658.0,-34357.0,113246.0,-0.87,-17.89,71.83,41.08,158.0,-0.30,1.15,33.60,XY24,NTT,ENGINEERING
53,RAJESHEXPO,518.0,1867.17,68.0,L-SC,6.12,53660.0,-83517.0,83613.0,1.25,-60.88,155.82,0.07,267.0,-1.00,0.39,33.45,OX40N,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,43.52
2,50,73.90


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.49
LC    34.46
MC    23.05
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.96
X40      15.27
X40N     12.98
XR       10.07
XY25      9.77
AR        8.62
OX40N     8.21
X200      1.73
MH        1.71
X5K       1.41
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.38
H-LC    26.27
H-MC    20.11
M-SC    13.54
M-LC     7.13
M-MC     2.63
L-SC     1.57
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.40,-14.30,79.29
FMCG,11.92,-3.49,39.70
FINANCE,9.59,-16.80,62.55
BANKS,7.85,-11.42,62.72
MISC,6.11,-13.10,77.66
PAINTS,5.92,-13.72,39.17
ELECTRICAL,5.57,-7.57,66.86
AC,3.60,2.25,30.86
INSURANCE,3.40,-7.95,44.97


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2870859.0,22
AR,1312522.0,9
XR,1231238.0,13
X40,871205.0,10
X40N,630489.0,10
OX40N,518214.0,10
XY25,409575.0,7
SR,246821.0,2
X5K,123518.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3355840.0,28
H-LC,1386015.0,15
M-SC,1385863.0,17
H-MC,1193732.0,15
M-LC,400586.0,5
M-MC,316122.0,2
L-SC,245566.0,3
L-MC,59930.0,1
L-LC,38714.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1041271.0      6
M-SC       XY24       799346.0      7
H-SC       AR         781432.0      5
           XR         678589.0      6
H-LC       X40        581061.0      5
H-MC       XY24       484831.0      4
H-LC       AR         369601.0      2
M-MC       XY24       316122.0      2
H-SC       X40N       298271.0      4
           SR         246821.0      2
           OX40N      246286.0      4
H-LC       X40N       222596.0      4
H-MC       X40        219330.0      4
           XY25       187098.0      2
M-LC       XY24       163664.0      2
L-SC       XR         161953.0      2
M-SC       AR         161489.0      2
           OX40N      123556.0      4
M-LC       X5K        123518.0      1
M-SC       XR         117214.0      2
           XY25       113444.0      1
H-MC       XR         110581.0      1
           X40N       109622.0      2
M-LC       XR         102971.0      1
H-LC       X200        87246.0      1
L-SC       OX40N       83613.0      1
M-SC       X40         70814.0      1
H-LC       XY24        65625.0      1
H-MC       OX40N       64759.0      1
H-SC       MH          63170.0      1
L-MC       XR          59930.0      1
H-LC       XY25        59886.0      2
L-LC       XY25        38714.0      1
H-MC       MH          17511.0      1
M-LC       XY25        10433.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
